In [32]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import os
import cv2

In [33]:
prototxtPath=os.path.sep.join([r'H:\face mask detector\face_detector','deploy.prototxt'])
wieghtsPath=os.path.sep.join([r'H:\face mask detector\face_detector','res10_300x300_ssd_iter_140000.caffemodel'])

In [34]:
prototxtPath

'H:\\face mask detector\\face_detector\\deploy.prototxt'

In [35]:
wieghtsPath

'H:\\face mask detector\\face_detector\\res10_300x300_ssd_iter_140000.caffemodel'

In [36]:
net=cv2.dnn.readNet(prototxtPath,wieghtsPath)

In [37]:
model=load_model(r'H:\face mask detector\model.h5')

In [56]:
# for i in range(1,3):
#     c=str(i)
#     imagename='\example_0'+c+'.png'
image=cv2.imread(r'H:\face mask detector\examples\example_03.png')

In [57]:
image

array([[[152, 153, 148],
        [149, 150, 146],
        [149, 150, 146],
        ...,
        [147, 150, 148],
        [145, 148, 146],
        [146, 149, 147]],

       [[147, 149, 146],
        [148, 151, 146],
        [150, 152, 147],
        ...,
        [150, 152, 149],
        [146, 149, 147],
        [145, 148, 146]],

       [[145, 148, 147],
        [147, 150, 147],
        [150, 154, 149],
        ...,
        [153, 153, 151],
        [150, 152, 150],
        [147, 151, 149]],

       ...,

       [[131, 130, 134],
        [129, 129, 132],
        [129, 129, 130],
        ...,
        [173, 177, 181],
        [176, 182, 186],
        [180, 187, 191]],

       [[133, 131, 136],
        [127, 128, 130],
        [130, 133, 133],
        ...,
        [179, 184, 187],
        [181, 187, 190],
        [183, 191, 194]],

       [[130, 130, 135],
        [127, 128, 133],
        [127, 129, 132],
        ...,
        [184, 189, 192],
        [184, 190, 193],
        [186, 193, 196]]

In [58]:
image.shape

(400, 600, 3)

In [59]:
(h,w)=image.shape[:2]

In [60]:
(h,w)

(400, 600)

In [61]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [62]:
blob

array([[[[ 46.,  44.,  46., ...,  47.,  44.,  41.],
         [ 43.,  45.,  44., ...,  43.,  46.,  43.],
         [ 43.,  44.,  41., ...,  41.,  47.,  46.],
         ...,
         [ 26.,  27.,  34., ...,  45.,  63.,  73.],
         [ 26.,  27.,  32., ...,  52.,  69.,  76.],
         [ 25.,  25.,  33., ...,  62.,  77.,  80.]],

        [[-26., -27., -24., ..., -26., -27., -29.],
         [-27., -25., -26., ..., -27., -26., -27.],
         [-26., -25., -25., ..., -29., -25., -26.],
         ...,
         [-48., -45., -35., ..., -29.,  -8.,   6.],
         [-47., -44., -37., ..., -22.,  -1.,  10.],
         [-48., -46., -36., ..., -12.,   7.,  14.]],

        [[ 24.,  22.,  22., ...,  24.,  24.,  23.],
         [ 24.,  24.,  22., ...,  22.,  25.,  25.],
         [ 25.,  25.,  22., ...,  20.,  26.,  26.],
         ...,
         [ 10.,   9.,  20., ...,  28.,  48.,  62.],
         [ 10.,  10.,  17., ...,  37.,  56.,  67.],
         [ 11.,  10.,  20., ...,  46.,  64.,  71.]]]], dtype=float32)

In [63]:
blob.shape

(1, 3, 300, 300)

In [14]:
blob

array([[[[131., 131., 131., ...,  66.,  65.,  65.],
         [131., 131., 131., ...,  66.,  65.,  65.],
         [131., 131., 131., ...,  66.,  65.,  65.],
         ...,
         [ 65.,  65.,  67., ...,  16.,  14.,  14.],
         [ 65.,  66.,  67., ...,  14.,  13.,  11.],
         [ 65.,  66.,  67., ...,  13.,  10.,   8.]],

        [[ 58.,  58.,  58., ...,  -7.,  -8.,  -8.],
         [ 58.,  58.,  58., ...,  -7.,  -8.,  -8.],
         [ 58.,  58.,  58., ...,  -7.,  -8.,  -8.],
         ...,
         [ -8.,  -8.,  -6., ..., -57., -59., -59.],
         [ -8.,  -7.,  -6., ..., -59., -60., -62.],
         [ -8.,  -7.,  -6., ..., -60., -63., -65.]],

        [[112., 112., 112., ...,  47.,  46.,  46.],
         [112., 112., 112., ...,  47.,  46.,  46.],
         [112., 112., 112., ...,  47.,  46.,  46.],
         ...,
         [ 46.,  46.,  48., ...,  -3.,  -5.,  -5.],
         [ 46.,  47.,  48., ...,  -5.,  -6.,  -8.],
         [ 46.,  47.,  48., ...,  -6.,  -9., -11.]]]], dtype=float32)

In [64]:
net.setInput(blob)
detections=net.forward()

In [65]:
detections[0,0,0,3:7]

array([0.28936234, 0.18173476, 0.4329286 , 0.4609872 ], dtype=float32)

In [66]:
#loop over detections
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
#     print(confidence)
    if(confidence>0.5):
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure that the dimensions of box are under the frame window
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX),min(h-1,endY))
        
        #extract the face roi , convert it from BGR to RBG channel, resize it to (224,224) and preprocess it
        
        face=image[startY:endY,startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color accordingly
        label='Mask' if (mask>withoutMask) else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #show the probability of the mask
        label="{}:{:.2f}%".format(label,max(mask,withoutMask)*100)
        imagecopy=image.copy()
        
        #display the label and the bounding frame
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
cv2.imshow('output',image)
cv2.waitKey(0)
cv2.destroyAllWindows()
    